In [36]:
import tensorflow as tf
import numpy as np 
import torch
import random
import math
import copy

In [2]:
def uniform_random(n, samples):
    indis = [0 for i in range(samples)]
    for i in range (samples):
        indis[i] = random.randint(int(i*2**(n*n)/samples), int((i+1)*2**(n*n)/samples))
    return indis


In [3]:
def indices ( num_indices , _range ):
    _list = [[]]
    return ( _index_gen_helper ( num_indices , _range , _list ))
def _index_gen_helper ( num_indices , _range , _list ):
    _ret = []

    for i in _list :
        for j in range ( _range ):
            _temp = i. copy ()
            _temp . extend ([ j ])
            _ret . append ( _temp )
    if num_indices == 1:
        return _ret
    else :
        return _index_gen_helper ( num_indices -1 , _range , _ret )

In [4]:
def instant_indices(num_independent_indices, i):
    inds=[0 for i in range (num_independent_indices)]
    for j in range (0, num_independent_indices):
        inds[j] = i%2
        i//=2
    return inds

print(instant_indices(4,10))

[0, 1, 0, 1]


In [5]:
#A and B are the spin up and spin down tensors

def net_2x2(A,B):
    AB=[A, B]
    C = [0 for i in range (2**4)]
    count=0
    for i ,j ,k ,l in indices(4,2):  
        C[count]=torch.einsum('zijab, zjicd, zklba, zlkdc->z', AB[i], AB[j], AB[k], AB[l])
        count+=1  
    ans=torch.stack(C)
    return tf.transpose(ans)

def batched_single_net_2x2(A,B, i):
    A.requires_grad = True
    B.requires_grad = True
    AB=[A, B]
    ans=torch.einsum('zijab, zjicd, zklba, zlkdc->z', AB[i[0]], AB[i[1]], AB[i[2]], AB[i[3]])
    return ans.reshape(A.shape[0],1)

def single_net_2x2(A,B, i):
#     A.requires_grad = True
#     B.requires_grad = True
    AB=[A, B]
    ans=torch.einsum('ijab, jicd, klba, lkdc', AB[i[0]], AB[i[1]], AB[i[2]], AB[i[3]])
    return ans


A = torch.rand((2,2,2,2))
B = torch.rand((2,2,2,2))
C= torch.rand((2,2,2,2))
D= torch.rand((2,2,2,2))
E = torch.rand((2,2,2,2))
F=torch.rand((2,2,2,2))
G=torch.rand((2,2,2,2))

x=[A,C,E,A,A,A,A,A,A,A]
y=[B,F,G,B,B,B,B,B,B,B]

netA=torch.stack(x)
netB=torch.stack(y)

# print(net_2x2(netA,netB))
# print()

num = random.randint(0, 2**(2**2))
config = instant_indices(2**2, num)
print(single_net_2x2(A,B,config))
# print(z)
# z.mean().backward()
# print(netA.grad.data)




tensor(22.6747)


In [6]:
def net_3x3(A,B):
    AB=[A,B]
    C = [0 for i in range (2**9)]
    count=0
    for i ,j ,k ,l ,m ,n ,o ,p ,q in indices(9, 2):    
        C[count]=torch.einsum('zabjk, zcamn, zbcpq, zdelj, zfdom, zefrp, zghkl, zigno, zhiqr->z', AB[i], AB[j], AB[k], AB[l], AB[m], AB[n], AB[o], AB[p], AB[q])
        count+=1
    ans=torch.stack(C)
    return tf.transpose(ans)

def batched_single_net_3x3(A,B, i):
    A.requires_grad = True
    B.requires_grad = True
    AB=[A,B]  
    ans=torch.einsum('zabjk, zcamn, zbcpq, zdelj, zfdom, zefrp, zghkl, zigno, zhiqr->z', AB[i[0]], AB[i[1]], AB[i[2]], AB[i[3]], AB[i[4]], AB[i[5]], AB[i[6]], AB[i[7]], AB[i[8]])
    return ans.reshape(A.shape[0],1)

def single_net_3x3(A,B, i):
#     A.requires_grad = True
#     B.requires_grad = True
    AB=[A,B]  
    ans=torch.einsum('abjk, camn, bcpq, delj, fdom, efrp, ghkl, igno, hiqr', AB[i[0]], AB[i[1]], AB[i[2]], AB[i[3]], AB[i[4]], AB[i[5]], AB[i[6]], AB[i[7]], AB[i[8]])
    return ans

# A = torch.rand((2,2,2,2))
# B = torch.rand((2,2,2,2))
# C= torch.rand((2,2,2,2))
# D= torch.rand((2,2,2,2))
# E = torch.rand((2,2,2,2))
# F=torch.rand((2,2,2,2))
# G=torch.rand((2,2,2,2))

# x=[A,C,E,A,A]
# y=[B,F,G,B,B]

# netA=torch.stack(x)
# netB=torch.stack(y)        
# print(net_3x3(netA,netB))

sample = instant_indices(9,20)
print(single_net_3x3(A,B,sample))


tensor(1028.6807)


In [7]:
def net_4x4(A,B, samples):
    AB=[A,B]
    C = [0 for i in range (samples)]
    count=0
    indis = uniform_random(4, samples)
    for i in (indis):
        CI = instant_indices(16,i) #current index
        X = torch.einsum('zabeh, zcail, zdcmp, zbdqt-> zehilmpqt', AB[CI[0]], AB[CI[1]], AB[CI[2]], AB[CI[3]])
        Y = torch.einsum('zabfe, zcaji, zdcnm, zbdrq-> zfejinmrq', AB[CI[4]], AB[CI[5]], AB[CI[6]], AB[CI[7]])
        Z = torch.einsum('zabgf, zcakj, zdcon, zbdsr-> zgfkjonsr', AB[CI[8]], AB[CI[9]], AB[CI[10]], AB[CI[11]])
        W = torch.einsum('zabhg, zcalk, zdcpo, zbdts-> zhglkpots', AB[CI[12]], AB[CI[13]], AB[CI[14]], AB[CI[15]])
        C[count] = torch.einsum('zehilmpqt, zfejinmrq, zgfkjonsr, zhglkpots->z', X, Y, Z, W)
        count+=1
    ans=torch.stack(C)
    return tf.transpose(ans)

def batched_single_net_4x4(A,B, CI):
    A.requires_grad = True
    B.requires_grad = True
    AB=[A,B]
    X = torch.einsum('zabeh, zcail, zdcmp, zbdqt-> zehilmpqt', AB[CI[0]], AB[CI[1]], AB[CI[2]], AB[CI[3]])
    Y = torch.einsum('zabfe, zcaji, zdcnm, zbdrq-> zfejinmrq', AB[CI[4]], AB[CI[5]], AB[CI[6]], AB[CI[7]])
    Z = torch.einsum('zabgf, zcakj, zdcon, zbdsr-> zgfkjonsr', AB[CI[8]], AB[CI[9]], AB[CI[10]], AB[CI[11]])
    W = torch.einsum('zabhg, zcalk, zdcpo, zbdts-> zhglkpots', AB[CI[12]], AB[CI[13]], AB[CI[14]], AB[CI[15]])
    ans = torch.einsum('zehilmpqt, zfejinmrq, zgfkjonsr, zhglkpots->z', X, Y, Z, W)
    return ans.reshape(A.shape[0],1)

def single_net_4x4(A,B, CI):
#     A.requires_grad = True
#     B.requires_grad = True
    AB=[A,B]
    X = torch.einsum('abeh, cail, dcmp, bdqt-> ehilmpqt', AB[CI[0]], AB[CI[1]], AB[CI[2]], AB[CI[3]])
    Y = torch.einsum('abfe, caji, dcnm, bdrq-> fejinmrq', AB[CI[4]], AB[CI[5]], AB[CI[6]], AB[CI[7]])
    Z = torch.einsum('abgf, cakj, dcon, bdsr-> gfkjonsr', AB[CI[8]], AB[CI[9]], AB[CI[10]], AB[CI[11]])
    W = torch.einsum('abhg, calk, dcpo, bdts-> hglkpots', AB[CI[12]], AB[CI[13]], AB[CI[14]], AB[CI[15]])
    ans = torch.einsum('ehilmpqt, fejinmrq, gfkjonsr, hglkpots', X, Y, Z, W)
    return ans

# A = torch.rand((2,2,2,2))
# B = torch.rand((2,2,2,2))
# C= torch.rand((2,2,2,2))
# D= torch.rand((2,2,2,2))
# E = torch.rand((2,2,2,2))
# F=torch.rand((2,2,2,2))
# G=torch.rand((2,2,2,2))

# x=[A,C,E,A,A]
# y=[B,F,G,B,B]

# netA=torch.stack(x)
# netB=torch.stack(y)  
# print(net_4x4(netA,netB,100))
# print()
sample = instant_indices(16, 100)
print(single_net_4x4(A,B,sample))

tensor(231698.5938)


In [8]:
def net_5x5(A,B, samples):
    AB=[A,B]
    C = [0 for i in range (samples)]
    count=0
    indis = uniform_random(5, samples)
    for i in (indis):
        CI = instant_indices(25,i) #current index
        X = torch.einsum('zabfg, zcahi, zdcjk, zedlm, zbeno -> zfghijklmno', AB[CI[0]], AB[CI[1]], AB[CI[2]], AB[CI[3]], AB[CI[4]]) 
        Y = torch.einsum('zabpf, zcaqh, zdcrj, zedsl, zbetn -> zpfqhrjsltn', AB[CI[5]], AB[CI[6]], AB[CI[7]], AB[CI[8]], AB[CI[9]])
        XY = torch.einsum('zfghijklmno, zpfqhrjsltn -> zgpiqkrmson', X, Y)
        Z = torch.einsum('zabfp, zcahq, zdcjr, zedls, zbent -> zfphqjrlsnt' , AB[CI[10]], AB[CI[11]], AB[CI[12]], AB[CI[13]], AB[CI[14]])
        XYZ = torch.einsum('zgpiqkrmson, zfphqjrlsnt -> zgfihkjmlon', XY, Z)
        U = torch.einsum('zabpf, zcaqh, zdcrj, zedsl, zbetn -> zpfqhrjsltn', AB[CI[15]], AB[CI[16]], AB[CI[17]], AB[CI[18]], AB[CI[19]])
        XYZU = torch.einsum('zgfihkjmlon, zpfqhrjsltn -> zgpiqkrmsot', XYZ, U)
        V = torch.einsum('zabgp, zcaiq, zdckr, zedms, zbeot -> zgpiqkrmsot', AB[CI[20]], AB[CI[21]], AB[CI[22]], AB[CI[23]], AB[CI[24]])
        C[count] = torch.einsum('zgpiqkrmsot, zgpiqkrmsot->z', XYZU, V)
        count+=1
    ans=torch.stack(C)
    return tf.transpose(ans)

def batched_single_net_5x5(A,B, CI):
    A.requires_grad = True
    B.requires_grad = True
    AB=[A,B]
    X = torch.einsum('zabfg, zcahi, zdcjk, zedlm, zbeno -> zfghijklmno', AB[CI[0]], AB[CI[1]], AB[CI[2]], AB[CI[3]], AB[CI[4]]) 
    Y = torch.einsum('zabpf, zcaqh, zdcrj, zedsl, zbetn -> zpfqhrjsltn', AB[CI[5]], AB[CI[6]], AB[CI[7]], AB[CI[8]], AB[CI[9]])
    XY = torch.einsum('zfghijklmno, zpfqhrjsltn -> zgpiqkrmson', X, Y)
    Z = torch.einsum('zabfp, zcahq, zdcjr, zedls, zbent -> zfphqjrlsnt' , AB[CI[10]], AB[CI[11]], AB[CI[12]], AB[CI[13]], AB[CI[14]])
    XYZ = torch.einsum('zgpiqkrmson, zfphqjrlsnt -> zgfihkjmlon', XY, Z)
    U = torch.einsum('zabpf, zcaqh, zdcrj, zedsl, zbetn -> zpfqhrjsltn', AB[CI[15]], AB[CI[16]], AB[CI[17]], AB[CI[18]], AB[CI[19]])
    XYZU = torch.einsum('zgfihkjmlon, zpfqhrjsltn -> zgpiqkrmsot', XYZ, U)
    V = torch.einsum('zabgp, zcaiq, zdckr, zedms, zbeot -> zgpiqkrmsot', AB[CI[20]], AB[CI[21]], AB[CI[22]], AB[CI[23]], AB[CI[24]])
    ans = torch.einsum('zgpiqkrmsot, zgpiqkrmsot->z', XYZU, V)
    return ans.reshape(A.shape[0],1)

def single_net_5x5(A,B, CI):
#     A.requires_grad = True
#     B.requires_grad = True
    AB=[A,B]
    X = torch.einsum('abfg, cahi, dcjk, edlm, beno -> fghijklmno', AB[CI[0]], AB[CI[1]], AB[CI[2]], AB[CI[3]], AB[CI[4]]) 
    Y = torch.einsum('abpf, caqh, dcrj, edsl, betn -> pfqhrjsltn', AB[CI[5]], AB[CI[6]], AB[CI[7]], AB[CI[8]], AB[CI[9]])
    XY = torch.einsum('fghijklmno, pfqhrjsltn -> gpiqkrmson', X, Y)
    Z = torch.einsum('abfp, cahq, dcjr, edls, bent -> fphqjrlsnt' , AB[CI[10]], AB[CI[11]], AB[CI[12]], AB[CI[13]], AB[CI[14]])
    XYZ = torch.einsum('gpiqkrmson, fphqjrlsnt -> gfihkjmlon', XY, Z)
    U = torch.einsum('abpf, caqh, dcrj, edsl, betn -> pfqhrjsltn', AB[CI[15]], AB[CI[16]], AB[CI[17]], AB[CI[18]], AB[CI[19]])
    XYZU = torch.einsum('gfihkjmlon, pfqhrjsltn -> gpiqkrmsot', XYZ, U)
    V = torch.einsum('abgp, caiq, dckr, edms, beot -> gpiqkrmsot', AB[CI[20]], AB[CI[21]], AB[CI[22]], AB[CI[23]], AB[CI[24]])
    ans = torch.einsum('gpiqkrmsot, gpiqkrmsot', XYZU, V)
    return ans

A = torch.rand((2,2,2,2))
B = torch.rand((2,2,2,2))
# C= torch.rand((2,2,2,2))
# D= torch.rand((2,2,2,2))
# E = torch.rand((2,2,2,2))
# F=torch.rand((2,2,2,2))
# G=torch.rand((2,2,2,2))

# x=[A,C,E,A,A]
# y=[B,F,G,B,B]

# netA=torch.stack(x)
# netB=torch.stack(y)  
# print(net_5x5(netA,netB,100))
# print()
sample = instant_indices(25, 10000)
print(single_net_5x5(A,B, sample))

tensor(44070472.)


In [9]:

def net_6x6(A, B, samples):
    AB=[A,B]
    C = [0 for i in range (samples)]
    count=0
    indis = uniform_random(6, samples)
    for i in (indis):
        CI = instant_indices(36,i) #current index
        X = torch.einsum('zabnh, zcaoi, zdcpj, zedqk, zferl, zbfsm -> znhoipjqkrlsm', AB[CI[0]], AB[CI[1]], AB[CI[2]], AB[CI[3]], AB[CI[4]], AB[CI[5]]) 
        Y = torch.einsum('zabtn, zcauo, zdcvp, zedwq, zfexr, zbfys -> ztnuovpwqxrys', AB[CI[6]], AB[CI[7]], AB[CI[8]], AB[CI[9]], AB[CI[10]], AB[CI[11]])
        XY = torch.einsum('znhoipjqkrlsm, ztnuovpwqxrys -> zhtiujvkwlxmy', X, Y)
        Z = torch.einsum('zabnt, zcaou, zdcpv, zedqw, zferx, zbfsy -> zntoupvqwrxsy', AB[CI[12]], AB[CI[13]], AB[CI[14]], AB[CI[15]], AB[CI[16]], AB[CI[17]])
        XYZ = torch.einsum('zhtiujvkwlxmy, zntoupvqwrxsy -> zhniojpkqlrms', XY, Z)
        U = torch.einsum('zabtn, zcauo, zdcvp, zedwq, zfexr, zbfys -> ztnuovpwqxrys', AB[CI[18]], AB[CI[19]], AB[CI[20]], AB[CI[21]], AB[CI[22]], AB[CI[23]])
        XYZU = torch.einsum('zhniojpkqlrms, ztnuovpwqxrys -> zhtiujvkwlxmy', XYZ, U)
        V = torch.einsum('zabnt, zcaou, zdcpv, zedqw, zferx, zbfsy -> zntoupvqwrxsy', AB[CI[24]], AB[CI[25]], AB[CI[26]], AB[CI[27]], AB[CI[28]], AB[CI[29]])
        XYZUV = torch.einsum('zhtiujvkwlxmy, zntoupvqwrxsy -> zhniojpkqlrms', XYZU, V)
        W = torch.einsum('zabhn, zcaio, zdcjp, zedkq, zfelr, zbfms -> zhniojpkqlrms', AB[CI[30]], AB[CI[31]], AB[CI[32]], AB[CI[33]], AB[CI[34]], AB[CI[35]])
        C[count] = torch.einsum('zhniojpkqlrms, zhniojpkqlrms->z', XYZUV, W)
        count+=1
    ans=torch.stack(C)
    return tf.transpose(ans)

def batched_single_net_6x6(A, B, CI):
    A.requires_grad = True
    B.requires_grad = True
    AB=[A,B] 
    X = torch.einsum('zabnh, zcaoi, zdcpj, zedqk, zferl, zbfsm -> znhoipjqkrlsm', AB[CI[0]], AB[CI[1]], AB[CI[2]], AB[CI[3]], AB[CI[4]], AB[CI[5]]) 
    Y = torch.einsum('zabtn, zcauo, zdcvp, zedwq, zfexr, zbfys -> ztnuovpwqxrys', AB[CI[6]], AB[CI[7]], AB[CI[8]], AB[CI[9]], AB[CI[10]], AB[CI[11]])
    XY = torch.einsum('znhoipjqkrlsm, ztnuovpwqxrys -> zhtiujvkwlxmy', X, Y)
    Z = torch.einsum('zabnt, zcaou, zdcpv, zedqw, zferx, zbfsy -> zntoupvqwrxsy', AB[CI[12]], AB[CI[13]], AB[CI[14]], AB[CI[15]], AB[CI[16]], AB[CI[17]])
    XYZ = torch.einsum('zhtiujvkwlxmy, zntoupvqwrxsy -> zhniojpkqlrms', XY, Z)
    U = torch.einsum('zabtn, zcauo, zdcvp, zedwq, zfexr, zbfys -> ztnuovpwqxrys', AB[CI[18]], AB[CI[19]], AB[CI[20]], AB[CI[21]], AB[CI[22]], AB[CI[23]])
    XYZU = torch.einsum('zhniojpkqlrms, ztnuovpwqxrys -> zhtiujvkwlxmy', XYZ, U)
    V = torch.einsum('zabnt, zcaou, zdcpv, zedqw, zferx, zbfsy -> zntoupvqwrxsy', AB[CI[24]], AB[CI[25]], AB[CI[26]], AB[CI[27]], AB[CI[28]], AB[CI[29]])
    XYZUV = torch.einsum('zhtiujvkwlxmy, zntoupvqwrxsy -> zhniojpkqlrms', XYZU, V)
    W = torch.einsum('zabhn, zcaio, zdcjp, zedkq, zfelr, zbfms -> zhniojpkqlrms', AB[CI[30]], AB[CI[31]], AB[CI[32]], AB[CI[33]], AB[CI[34]], AB[CI[35]])
    ans = torch.einsum('zhniojpkqlrms, zhniojpkqlrms->z', XYZUV, W)
    return ans.reshape(A.shape[0],1)

def single_net_6x6(A, B, CI):
#     A.requires_grad = True
#     B.requires_grad = True
    AB=[A,B] 
    X = torch.einsum('abnh, caoi, dcpj, edqk, ferl, bfsm -> nhoipjqkrlsm', AB[CI[0]], AB[CI[1]], AB[CI[2]], AB[CI[3]], AB[CI[4]], AB[CI[5]]) 
    Y = torch.einsum('abtn, cauo, dcvp, edwq, fexr, bfys -> tnuovpwqxrys', AB[CI[6]], AB[CI[7]], AB[CI[8]], AB[CI[9]], AB[CI[10]], AB[CI[11]])
    XY = torch.einsum('nhoipjqkrlsm, tnuovpwqxrys -> htiujvkwlxmy', X, Y)
    Z = torch.einsum('abnt, caou, dcpv, edqw, ferx, bfsy -> ntoupvqwrxsy', AB[CI[12]], AB[CI[13]], AB[CI[14]], AB[CI[15]], AB[CI[16]], AB[CI[17]])
    XYZ = torch.einsum('htiujvkwlxmy, ntoupvqwrxsy -> hniojpkqlrms', XY, Z)
    U = torch.einsum('abtn, cauo, dcvp, edwq, fexr, bfys -> tnuovpwqxrys', AB[CI[18]], AB[CI[19]], AB[CI[20]], AB[CI[21]], AB[CI[22]], AB[CI[23]])
    XYZU = torch.einsum('hniojpkqlrms, tnuovpwqxrys -> htiujvkwlxmy', XYZ, U)
    V = torch.einsum('abnt, caou, dcpv, edqw, ferx, bfsy -> ntoupvqwrxsy', AB[CI[24]], AB[CI[25]], AB[CI[26]], AB[CI[27]], AB[CI[28]], AB[CI[29]])
    XYZUV = torch.einsum('htiujvkwlxmy, ntoupvqwrxsy -> hniojpkqlrms', XYZU, V)
    W = torch.einsum('abhn, caio, dcjp, edkq, felr, bfms -> hniojpkqlrms', AB[CI[30]], AB[CI[31]], AB[CI[32]], AB[CI[33]], AB[CI[34]], AB[CI[35]])
    ans = torch.einsum('hniojpkqlrms, hniojpkqlrms', XYZUV, W)
    return ans

A = torch.rand((2,2,2,2))
B = torch.rand((2,2,2,2))
C= torch.rand((2,2,2,2))
D= torch.rand((2,2,2,2))
E = torch.rand((2,2,2,2))
F=torch.rand((2,2,2,2))
G=torch.rand((2,2,2,2))

x=[A,C,E,A,A]
y=[B,F,G,B,B]

netA=torch.stack(x)
netB=torch.stack(y)  
# print(net_6x6(netA,netB,100))
# print()

sample = instant_indices(36, 8929789328)
print(single_net_6x6(A,B, sample))

tensor(6.3709e+10)


In [10]:
#A is a batch of spin up tensors
#B is a batch of spin down tensors

def wave_func (A, B, net, samples):
    if(net==2):
        return net_2x2(A, B)
    elif(net==3):
        return net_3x3(A, B)
    elif(net==4):
        return net_4x4(A, B, samples)
    elif(net==5):
        return net_5x5(A, B, samples)
    elif(net==6):
        return net_6x6(A, B, samples)
    else:
        return 0
        
def single_wave_func (A, B, net, sample):
    if(net==2):
        return single_net_2x2(A, B, sample)
    elif(net==3):
        return single_net_3x3(A, B, sample)
    elif(net==4):
        return single_net_4x4(A, B, sample)
    elif(net==5):
        return single_net_5x5(A, B, sample)
    elif(net==6):
        return single_net_6x6(A, B, sample)
    else:
        return 0

In [53]:
def Monte_Carlo(A, B, net):
    num = random.randint(0, 2**(net**2))
    config = instant_indices(net**2, num)
    print(config)
    Ca = single_wave_func (A, B, net, config)
    unnorma = Ca**2
    
    for i in range (net**2):
        temp_config = copy.copy(config)
        temp_config[i] = (1-temp_config[i])
        Cb = single_wave_func(A, B, net, temp_config)
        unnormb = Cb**2
        r = unnormb/unnorma
        x = random.uniform(0,1)
        
        if (r > x):

            config = copy.copy(temp_config)
            Ca = Cb
        else:
 
    return config

A = torch.rand((2,2,2,2))
B = torch.rand((2,2,2,2))

print(Monte_Carlo(A, B, 6))

[0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1]
tensor(0.8464)
0.0380399434753389

tensor(0.7130)
0.5202684732895764

tensor(0.8409)
0.5794690372474665

tensor(0.9906)
0.12950602908930542

tensor(1.1889)
0.15623345269249955

tensor(1.0017)
0.6559790591178493

tensor(1.1622)
0.44807434183507366

tensor(0.9755)
0.0754911815492949

tensor(1.1420)
0.6857124265022787

tensor(1.3865)
0.3130865238621916

tensor(1.6310)
0.36835229372997824

tensor(1.3732)
0.1506875886044523

tensor(1.6166)
0.12553181044622153

tensor(1.3902)
0.12655454162044555

tensor(1.6688)
0.9775357269886427

tensor(1.4291)
0.16682832899745503

tensor(1.7348)
0.41161017953071

tensor(1.4758)
0.6398127942710324

tensor(1.7973)
0.3895295568534036

tensor(2.0790)
0.7454533006559139

tensor(1.7042)
0.6332413875727487

tensor(2.0018)
0.4365746462727961

tensor(1.7033)
0.7585019678904937

tensor(1.4575)
0.6268737278650064

tensor(1.2064)
0.11496841953077896

tensor(1.4

In [39]:
Cb = instant_indices(4**2, 20)
print(Cb)
Cb[0] = (1-int(Cb[0]))
print(Cb)
print(random.randint(0, 2**(2*2)))
print(random.uniform(0,1))
for i in range (10):
    print(i)

[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
6
0.28023495644802143
0
1
2
3
4
5
6
7
8
9
